# Content
1. [Alignment](#alignment)
2. [SNP calling](#snpcalling)

## 1. Alignment <a id='alignment'></a>

Align reads to target sequences

In [ ]:
bwa mem -t 12 -M targetSequence.fasta \
read_1_repaired.fastq.gz
read_2_repaired.fastq.gz \
> sample.sam

Convert alignment format

In [ ]:
samtools view -bShq 20 sample.sam > sample.bam

Sort alignment

In [ ]:
samtools sort -m 4G -@ 8 -o sample-sort.bam sample.bam

Add read group information

In [ ]:
java -jar picard.jar \
AddOrReplaceReadGroups \
I=sample-sort.bam O=sample-addRG.bam \
RGID=lane+flowcell RGLB=sample RGPL=illumina RGPU=lane+flowcell+adaptor RGSM=sample

Mark duplicates

In [ ]:
java -jar picard.jar MarkDuplicates \
I=sample-addRG.bam \
O=sample-markDup.bam \
M=sample-markDup.txt \
MAX_RECORDS_IN_RAM=300000 \
USE_JDK_DEFLATER=TRUE \
USE_JDK_INFLATER=TRUE \
MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=2000 \
REMOVE_DUPLICATES=TRUE

Index alignment

In [ ]:
java -jar picard.jar BuildBamIndex \
I=sample

Index target sequences

In [ ]:
samtools faidx targetSequence.fasta

Create sequence directory

In [ ]:
java -jar picard.jar CreateSequenceDictionary \
R=targetSequence.fasta \
O=targetSequence.dict

Create realigner target

In [ ]:
java -jar GenomeAnalysisTK.jar \
-T RealignerTargetCreator -nt 12 \
-R targetSequence.fasta \
-I list-of-sample-markDup.bam-files.list \
-o batch.intervals

Realign indel

In [ ]:
java -Xmx8G -Djava.io.tmpdir=/tmp \
-jar GenomeAnalysisTK.jar \
-T IndelRealigner \
-R targetSequence.fasta \
-targetIntervals batch.intervals \
-I list-of-sample-markDup.bam-files.list \
-nWayOut list-of-output-paths.map

Inspect ancient DNA damage

In [ ]:
mapDamage -i sample-markDup.bam \
-r targetSequence.fasta \
-d sample \
--rescale --merge-reference-sequences -y 0.2 --threads 6

Trim sequences to remove damaged bases (e.g. 1bp from both ends)

In [ ]:
fastp \
--in1 read_1_fastuniq.tagged_filter.fastq.gz \
--in2 read_2_fastuniq.tagged_filter.fastq.gz \
--out1 read_1_trim1.fq.gz --out2 read_2_trim1.fq.gz \
-t 1 -F 1 -q 20

Check alignment quality

In [ ]:
qualimap bamqc \
-bam sample-realigned.bam \
-outdir sample \
-outfile sample -outformat html \
-ip -sd -nt 12

## 2. SNP calling <a id='snpcalling'></a>

Generate alignment likelihoods and call SNPs

In [ ]:
bcftools mpileup -Ou --skip-indels \
-a FORMAT/DP,INFO/DPR,FORMAT/DV,FORMAT/DP4,FORMAT/DPR,FORMAT/SP,FORMAT/AD,FORMAT/ADF,FORMAT/ADR,INFO/AD,INFO/ADF,INFO/ADR \
-f targetSequence.fasta \
-b bam.list | \
bcftools call -mv -Ov -o snp.vcf

Filter SNPs

In [ ]:
vcftools --vcf snp.vcf --minQ 30 --recode --recode-INFO-all --out snp-1
vcftools --vcf snp-1.vcf --min-meanDP 30 --max-meanDP 150 --recode --recode-INFO-all --out snp-2
vcftools --vcf snp-2.vcf --maf 0.02 --recode --recode-INFO-all --out snp-3
vcftools --vcf snp-3.vcf --max-missing 0.95 --recode --recode-INFO-all --out snp-4

Call one SNP per locus

In [ ]:
perl randSnps.pl \
< snp-4.vcf > \
snp_randSnps.vcf

Screen SNPs for linkage disequilibrium

In [ ]:
plink --vcf snp_randSNPs.vcf --allow-no-sex --allow-extra-chr --indep-pairwise 50 10 0.9
plink --vcf snp_randSNPs.vcf --extract plink.prune.in --allow-extra-chr --make-bed --out snp_randSNPs-prune
plink --bfile snp_randSNPs-prune --allow-extra-chr --recode vcf --out snp_randSNPs-plink

Screen SNPs for neutrality

In [ ]:
BayeScan2.1_linux64bits \
snp_randSNPs-plink \
-od path/to/output